# Inserting Customer table into Mysql so i can fetch it later

In [1]:
import pandas as pd
import mysql.connector

sqlConfig = {
    'host': 'localhost',
    'user': 'root',
    'password': 'root',
    'database': 'CustomerData'
}

# Load CSV file into a pandas DataFrame
csv_file_path = r'stores_sales_forecasting.xlsx'
df = pd.read_excel(csv_file_path)
df = df[['Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code', 'Region']]
df = df.rename(columns={'Postal Code': 'Postal_Code','Customer ID':'Customer_ID','Customer Name':'Customer_Name'})
df


,Customer_ID,Customer_Name,Segment,Country,City,State,Postal_Code,Region
0,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South
1,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South
2,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South
3,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West
4,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West
...,...,...,...,...,...,...,...,...
2116,CM-12655,Corinna Mitchell,Home Office,United States,Houston,Texas,77041,Central
2117,HE-14800,Harold Engle,Corporate,United States,Newark,Delaware,19711,East
2118,SW-20455,Shaun Weien,Consumer,United States,Lafayette,Louisiana,70506,South
2119,TB-21400,Tom Boeckenhauer,Consumer,United States,Miami,Florida,33180,South


In [2]:
conn = mysql.connector.connect(**sqlConfig)
cursor = conn.cursor()

create_table_query = """
CREATE TABLE IF NOT EXISTS customers (
    Customer_ID VARCHAR(255),
    Customer_Name VARCHAR(255),
    Segment VARCHAR(50),
    Country VARCHAR(100),
    City VARCHAR(100),
    State VARCHAR(100),
    Postal_Code VARCHAR(20),
    Region VARCHAR(100)
);
"""

cursor.execute(create_table_query)

for _, row in df.iterrows():
    print(tuple(row))
    insert_query = """
    INSERT INTO customers 
    (Customer_ID, Customer_Name, Segment, Country, City, State, Postal_Code, Region) 
    VALUES 
    (%s, %s, %s, %s, %s, %s, %s, %s)
    """
    cursor.execute(insert_query, tuple(row))
conn.commit()
cursor.close()
conn.close()

('CG-12520', 'Claire Gute', 'Consumer', 'United States', 'Henderson', 'Kentucky', 42420, 'South')
('CG-12520', 'Claire Gute', 'Consumer', 'United States', 'Henderson', 'Kentucky', 42420, 'South')
('SO-20335', "Sean O'Donnell", 'Consumer', 'United States', 'Fort Lauderdale', 'Florida', 33311, 'South')
('BH-11710', 'Brosina Hoffman', 'Consumer', 'United States', 'Los Angeles', 'California', 90032, 'West')
('BH-11710', 'Brosina Hoffman', 'Consumer', 'United States', 'Los Angeles', 'California', 90032, 'West')
('SF-20065', 'Sandra Flanagan', 'Consumer', 'United States', 'Philadelphia', 'Pennsylvania', 19140, 'East')
('EB-13870', 'Emily Burns', 'Consumer', 'United States', 'Orem', 'Utah', 84057, 'West')
('TB-21520', 'Tracy Blumstein', 'Consumer', 'United States', 'Philadelphia', 'Pennsylvania', 19140, 'East')
('TB-21520', 'Tracy Blumstein', 'Consumer', 'United States', 'Philadelphia', 'Pennsylvania', 19140, 'East')
('GH-14485', 'Gene Hale', 'Corporate', 'United States', 'Richardson', 'Texas

# Seperating Data for API Product Cost

In [3]:
csv_file_path = r'stores_sales_forecasting.xlsx'
df = pd.read_excel(csv_file_path)
df

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
3,6,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,90032,West,FUR-FU-10001487,Furniture,Furnishings,Eldon Expressions Wood and Plastic Desk Access...,48.8600,7,0.00,14.1694
4,11,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,90032,West,FUR-TA-10001539,Furniture,Tables,Chromcraft Rectangular Conference Tables,1706.1840,9,0.20,85.3092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2116,9963,CA-2015-168088,2015-03-19,2015-03-22,First Class,CM-12655,Corinna Mitchell,Home Office,United States,Houston,...,77041,Central,FUR-BO-10004218,Furniture,Bookcases,Bush Heritage Pine Collection 5-Shelf Bookcase...,383.4656,4,0.32,-67.6704
2117,9965,CA-2016-146374,2016-12-05,2016-12-10,Second Class,HE-14800,Harold Engle,Corporate,United States,Newark,...,19711,East,FUR-FU-10002671,Furniture,Furnishings,Electrix 20W Halogen Replacement Bulb for Zoom...,13.4000,1,0.00,6.4320
2118,9981,US-2015-151435,2015-09-06,2015-09-09,Second Class,SW-20455,Shaun Weien,Consumer,United States,Lafayette,...,70506,South,FUR-TA-10001039,Furniture,Tables,KI Adjustable-Height Table,85.9800,1,0.00,22.3548
2119,9990,CA-2014-110422,2014-01-21,2014-01-23,Second Class,TB-21400,Tom Boeckenhauer,Consumer,United States,Miami,...,33180,South,FUR-FU-10001889,Furniture,Furnishings,Ultra Door Pull Handle,25.2480,3,0.20,4.1028


In [16]:
df_api=df[['Order ID','Product ID','Product Name','Category','Sub-Category']]
df_api['Price']=df['Sales'] / df['Quantity']
df_api.to_csv("api_Data.csv",index=False)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_11372\4192088666.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_api['Price']=df['Sales'] / df['Quantity']


In [18]:
df_api

,Order ID,Product ID,Product Name,Category,Sub-Category,Price
0,CA-2016-152156,FUR-BO-10001798,Bush Somerset Collection Bookcase,Furniture,Bookcases,130.9800
1,CA-2016-152156,FUR-CH-10000454,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",Furniture,Chairs,243.9800
2,US-2015-108966,FUR-TA-10000577,Bretford CR4500 Series Slim Rectangular Table,Furniture,Tables,191.5155
3,CA-2014-115812,FUR-FU-10001487,Eldon Expressions Wood and Plastic Desk Access...,Furniture,Furnishings,6.9800
4,CA-2014-115812,FUR-TA-10001539,Chromcraft Rectangular Conference Tables,Furniture,Tables,189.5760
...,...,...,...,...,...,...
2116,CA-2015-168088,FUR-BO-10004218,Bush Heritage Pine Collection 5-Shelf Bookcase...,Furniture,Bookcases,95.8664
2117,CA-2016-146374,FUR-FU-10002671,Electrix 20W Halogen Replacement Bulb for Zoom...,Furniture,Furnishings,13.4000
2118,US-2015-151435,FUR-TA-10001039,KI Adjustable-Height Table,Furniture,Tables,85.9800
2119,CA-2014-110422,FUR-FU-10001889,Ultra Door Pull Handle,Furniture,Furnishings,8.4160


# Making seperate csv file for ETL to extract

In [21]:
df_csv=df[['Order ID','Order Date','Ship Date','Ship Mode','Product ID','Customer ID','Sales','Quantity','Discount','Profit']]
df_csv
df_csv.to_csv("Order_Info.csv",index=False)
df_csv

,Order ID,Order Date,Ship Date,Ship Mode,Product ID,Customer ID,Sales,Quantity,Discount,Profit
0,CA-2016-152156,2016-11-08,2016-11-11,Second Class,FUR-BO-10001798,CG-12520,261.9600,2,0.00,41.9136
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,FUR-CH-10000454,CG-12520,731.9400,3,0.00,219.5820
2,US-2015-108966,2015-10-11,2015-10-18,Standard Class,FUR-TA-10000577,SO-20335,957.5775,5,0.45,-383.0310
3,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,FUR-FU-10001487,BH-11710,48.8600,7,0.00,14.1694
4,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,FUR-TA-10001539,BH-11710,1706.1840,9,0.20,85.3092
...,...,...,...,...,...,...,...,...,...,...
2116,CA-2015-168088,2015-03-19,2015-03-22,First Class,FUR-BO-10004218,CM-12655,383.4656,4,0.32,-67.6704
2117,CA-2016-146374,2016-12-05,2016-12-10,Second Class,FUR-FU-10002671,HE-14800,13.4000,1,0.00,6.4320
2118,US-2015-151435,2015-09-06,2015-09-09,Second Class,FUR-TA-10001039,SW-20455,85.9800,1,0.00,22.3548
2119,CA-2014-110422,2014-01-21,2014-01-23,Second Class,FUR-FU-10001889,TB-21400,25.2480,3,0.20,4.1028
